Depositos para enviar, orderm de preferencia:
musical_matriz
musical_filal
em seguida onde houver mais estoque


onde days_available = 0 criar observação onde informa que há muito tempo sem estoque

In [1]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql
import math
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta
import numpy as np

load_dotenv()

ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host":HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

# Registra o tempo antes da execução
start_prog = time.time()

In [2]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

pd.reset_option('display.max_columns')


In [3]:
def condf(df, coluna, valor):
    """
    Consulta um DataFrame com base em uma coluna e valor específicos.

    Parâmetros:
    - df: DataFrame a ser consultado.
    - coluna: Nome da coluna para a condição de consulta.
    - valor: Valor desejado na coluna.

    Retorna:
    Um DataFrame contendo apenas as linhas que atendem à condição.
    """
    resultado = df[df[coluna] == valor]
    return resultado

def condf_date(df, coluna_data, data_pesquisada):
    """
    Consulta um DataFrame com base em uma coluna de datas.

    Parâmetros:
    - df: DataFrame a ser consultado.
    - coluna_data: Nome da coluna de datas.
    - data_pesquisada: Data desejada para a consulta.

    Retorna:
    Um DataFrame contendo apenas as linhas que correspondem à data pesquisada.
    """
    resultado = df[pd.to_datetime(df[coluna_data]).dt.date == data_pesquisada]
    return resultado


### Período a consultar

In [38]:
# Defina as datas de início e fim desejadas
data_inicio = datetime(2023, 11, 15).date()
data_fim = datetime(2023, 11, 27).date()
data_fim = data_fim + timedelta(days=1) # + 1 dia para pegar a data atual no DB

### Historico de estoque

In [39]:
# Buscando histórico de estoque na tabela
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM fulfillment_stock_hist WHERE created_at BETWEEN '{data_inicio}' AND '{data_fim}'"
    print(sql_query)
    df_stock = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")

except Exception as e:
    print(f"Erro ao consultar fulfillment_stock: {e}")

finally:
    if conn is not None:
        conn.close()

SELECT * FROM fulfillment_stock_hist WHERE created_at BETWEEN '2023-11-15' AND '2023-11-28'


C:\Users\couti\AppData\Local\Temp\ipykernel_1084\2214238412.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stock = pd.read_sql(sql_query, conn)


In [40]:
df_stock

,ml_inventory_id,available_quantity,created_at
0,DSGP06967,6,2023-11-23 11:43:27.867518
1,DSGP06979,7,2023-11-23 11:43:27.867518
2,JJSZ06277,48,2023-11-23 11:43:27.867518
3,DGCL82617,13,2023-11-23 11:43:27.867518
4,GCYF86172,12,2023-11-23 11:43:27.867518
...,...,...,...
10155,ERIM51807,0,2023-11-27 00:31:03.121363
10156,TDFV51283,0,2023-11-27 00:31:03.121363
10157,OYUK51882,15,2023-11-27 00:31:03.121363
10158,BLHH53768,4,2023-11-27 00:31:03.121363


In [7]:
# datas consultadas, dias em que um produto pode ou não estar disponível
df_stock['created_at'].value_counts().index.to_list()

[Timestamp('2023-11-20 01:00:00'),
 Timestamp('2023-11-19 01:00:00'),
 Timestamp('2023-11-18 01:00:00'),
 Timestamp('2023-11-17 01:00:00'),
 Timestamp('2023-11-16 01:00:00'),
 Timestamp('2023-11-15 01:00:00'),
 Timestamp('2023-11-23 11:43:27.867518'),
 Timestamp('2023-11-24 17:05:53.783044'),
 Timestamp('2023-11-25 00:31:02.644934')]

In [8]:
# Ordenando stock por data
df_stock = df_stock.sort_values(by='created_at', ascending=False)
df_stock['data'] = df_stock['created_at'].dt.date
df_stock = df_stock.drop(['created_at'], axis=1)

df_stock

,ml_inventory_id,available_quantity,data
9524,RKCI54992,0,2023-11-25
9098,OKDI95180,0,2023-11-25
9105,ZXJO91694,4,2023-11-25
9104,YLJM05104,2,2023-11-25
9103,PBFO04192,4,2023-11-25
...,...,...,...
8042,DWNN86592,1,2023-11-15
8041,OWZF76125,0,2023-11-15
8040,SDNY53354,2,2023-11-15
8039,KMCL67683,5,2023-11-15


In [9]:
## Cria coluna has_stock, se available_quantity <= 0, has_stock= False ##
df_stock = df_stock.assign(has_stock=lambda x: x["available_quantity"] > 0)
df_stock = df_stock.sort_values(by='data', ascending=False).reset_index(drop=True)
df_stock

,ml_inventory_id,available_quantity,data,has_stock
0,RKCI54992,0,2023-11-25,False
1,POTW78210,2,2023-11-25,True
2,IZBM09763,0,2023-11-25,False
3,IBGO88267,0,2023-11-25,False
4,BLHH53768,4,2023-11-25,True
...,...,...,...,...
9520,BLHH53768,4,2023-11-15,True
9521,RKCI54992,0,2023-11-15,False
9522,JJSZ06277,48,2023-11-15,True
9523,DSGP06979,7,2023-11-15,True


In [10]:
df_stock = df_stock.drop_duplicates()

In [11]:
condf(df_stock,'ml_inventory_id','DSGP06967')

,ml_inventory_id,available_quantity,data,has_stock
80,DSGP06967,6,2023-11-25,True
952,DSGP06967,6,2023-11-24,True
1691,DSGP06967,6,2023-11-23,True
2537,DSGP06967,6,2023-11-20,True
3809,DSGP06967,6,2023-11-19,True
4447,DSGP06967,6,2023-11-18,True
6348,DSGP06967,6,2023-11-17,True
7603,DSGP06967,6,2023-11-16,True
8257,DSGP06967,6,2023-11-15,True


#### Dias em que produto esteve disponível

In [12]:
## Contando dias em que produto esteve disponível 
days_available = df_stock.groupby('ml_inventory_id')['has_stock'].sum().reset_index()
days_available = days_available.rename(columns={'has_stock': 'days_available'})

In [13]:
condf(days_available,'ml_inventory_id','DSGP06967')

,ml_inventory_id,days_available
79,DSGP06967,9


In [14]:
# Unindo DFs
df_stock = df_stock.merge(days_available, on='ml_inventory_id', how='inner')

df_stock.shape

(5715, 5)

In [32]:
# data de hoje
# data_de_hoje = datetime.now().date() - timedelta(days=1)
# print(data_de_hoje)
data_de_hoje = datetime.now().date()
print(data_de_hoje)
df_stock['data'] = pd.to_datetime(df_stock['data'])

# Filtra apenas as linhas onde 'data' é igual à data de hoje
df_stock_today = df_stock[df_stock['data'].dt.date == data_de_hoje]
df_stock_today = df_stock_today.rename(columns={'available_quantity':'available_quantity_today'})
# df_stock_today = df_stock.drop(['has_stock'], axis=1)

2023-11-27


In [16]:
df_stock_today

,ml_inventory_id,available_quantity_today,data,has_stock,days_available


In [17]:
df_stock_today['days_available'].value_counts()

Series([], Name: count, dtype: int64)

### Buscando hitorico de orders no BD

In [18]:
# Buscando histórico de vendas na tabela ml_orders_hist para o período definido
try:
    conn = psycopg2.connect(**db_config)
    
    # Construa a consulta SQL com a condição de data
    sql_query = f"SELECT * FROM ml_orders_hist WHERE date_closed BETWEEN '{data_inicio}' AND '{data_fim}'"
    print(sql_query)
    # Execute a consulta e leia os dados em um DataFrame
    df_orders = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")

except Exception as e:
    print(f"Erro ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro ao consultar ml_orders_hist: {e}")

finally:
    if conn is not None:
        conn.close()

# filtros
df_orders = df_orders[df_orders['fulfilled'] == True]
df_orders = df_orders[df_orders['order_status'] == 'paid']
df_orders = df_orders[df_orders['payment_status'] == 'approved']
df_orders = df_orders.drop(['pack_id','date_approved','fulfilled','order_status','payment_status'], axis=1)
df_orders.rename(columns={'quantity': 'sales_quantity'}, inplace=True)


SELECT * FROM ml_orders_hist WHERE date_closed BETWEEN '2023-11-15' AND '2023-11-27'


C:\Users\couti\AppData\Local\Temp\ipykernel_1084\705596127.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_orders = pd.read_sql(sql_query, conn)


In [19]:
df_orders.sample()


,ml_code,category_id,variation_id,seller_sku,sales_quantity,title,shipping_id,date_closed
298,MLB3461718736,MLB72745,nan,FULLP10LNINJA20FT,1,Cabo Para Guitarra Violão Baixo Santo Angelo Ninja P10 X P10l 6 Metros,42806617088,2023-11-16 20:12:43


In [20]:
# Ordenando orders por data
df_orders = df_orders.sort_values(by='date_closed', ascending=False)
df_orders['data'] = df_orders['date_closed'].dt.date
df_orders = df_orders.drop(['date_closed'], axis=1)

print(df_orders.shape)
df_orders.head(3)

(802, 8)


,ml_code,category_id,variation_id,seller_sku,sales_quantity,title,shipping_id,data
1058,MLB1992541482,MLB278076,173552533267,FULLSG6687,3,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 85/15 Bronze Sg,42824583640,2023-11-21
1056,MLB1992567302,MLB429603,173585284763,FULLNI7AM,1,1 Par Baquetas De Bateria Liverpool Nirvana 5a Ou 5b Ou 7a,42824496718,2023-11-21
1051,MLB3493414145,MLB72752,nan,C4P04150,1,Correia Violão Guitarra Orleans Jacquard P04150 Ernie Ball,42824422982,2023-11-21


In [21]:
df_orders = df_orders.drop_duplicates()
df_orders.shape

(802, 8)

In [22]:
condf(df_orders,'ml_code','MLB1992541482')

,ml_code,category_id,variation_id,seller_sku,sales_quantity,title,shipping_id,data
1058,MLB1992541482,MLB278076,173552533267,FULLSG6687,3,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 85/15 Bronze Sg,42824583640,2023-11-21
942,MLB1992541482,MLB278076,173552533267,FULLSG6687,1,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 85/15 Bronze Sg,42822412452,2023-11-21
534,MLB1992541482,MLB278076,173552533264,FULLSG6684,1,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 85/15 Bronze Sg,42812680605,2023-11-18
270,MLB1992541482,MLB278076,173552533267,FULLSG6687,1,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 85/15 Bronze Sg,42806017696,2023-11-16
120,MLB1992541482,MLB278076,173552533264,FULLSG6684,2,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 85/15 Bronze Sg,42802756417,2023-11-15


#### Total de vendas por ml_code e seller_sku

In [23]:
# calcular total de vendas por ml_code e seller_sku no periodo
total_sales_by_filter = df_orders.groupby(['ml_code','seller_sku'])['sales_quantity'].sum().reset_index()
total_sales_by_filter.rename(columns={'sales_quantity': 'total_sales_quantity'}, inplace=True)

In [24]:
condf(total_sales_by_filter,'ml_code','MLB1992541482')

,ml_code,seller_sku,total_sales_quantity
39,MLB1992541482,FULLSG6684,3
40,MLB1992541482,FULLSG6687,5


In [25]:
# Acrescentando total de vendas ao DF
df_total_sales = pd.merge(df_orders, total_sales_by_filter, on=['ml_code','seller_sku'], how='inner')
df_total_sales.shape

(802, 9)

In [26]:
df_total_sales.head(3)

,ml_code,category_id,variation_id,seller_sku,sales_quantity,title,shipping_id,data,total_sales_quantity
0,MLB1992541482,MLB278076,173552533267,FULLSG6687,3,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 85/15 Bronze Sg,42824583640,2023-11-21,5
1,MLB1992541482,MLB278076,173552533267,FULLSG6687,1,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 85/15 Bronze Sg,42822412452,2023-11-21,5
2,MLB1992541482,MLB278076,173552533267,FULLSG6687,1,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 85/15 Bronze Sg,42806017696,2023-11-16,5


In [27]:
condf(df_total_sales,'ml_code','MLB1992541482')

,ml_code,category_id,variation_id,seller_sku,sales_quantity,title,shipping_id,data,total_sales_quantity
0,MLB1992541482,MLB278076,173552533267,FULLSG6687,3,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 85/15 Bronze Sg,42824583640,2023-11-21,5
1,MLB1992541482,MLB278076,173552533267,FULLSG6687,1,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 85/15 Bronze Sg,42822412452,2023-11-21,5
2,MLB1992541482,MLB278076,173552533267,FULLSG6687,1,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 85/15 Bronze Sg,42806017696,2023-11-16,5
648,MLB1992541482,MLB278076,173552533264,FULLSG6684,1,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 85/15 Bronze Sg,42812680605,2023-11-18,3
649,MLB1992541482,MLB278076,173552533264,FULLSG6684,2,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 85/15 Bronze Sg,42802756417,2023-11-15,3


In [28]:
df_total_sales = df_total_sales.drop(['sales_quantity','shipping_id','data'], axis=1)
df_total_sales = df_total_sales.drop_duplicates()

In [29]:
condf(df_total_sales,'ml_code','MLB1992541482')

,ml_code,category_id,variation_id,seller_sku,title,total_sales_quantity
0,MLB1992541482,MLB278076,173552533267,FULLSG6687,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 85/15 Bronze Sg,5
648,MLB1992541482,MLB278076,173552533264,FULLSG6684,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 85/15 Bronze Sg,3


Neste ponto temos o total de vendas de um anúncio por período e a quantidade de dias em que um produto esteve disponível.
precisamos juntar esses dados para calcular, para isso trarei as informações de produtos

In [30]:
print(df_total_sales.shape)
df_total_sales.sample()

(261, 6)


,ml_code,category_id,variation_id,seller_sku,title,total_sales_quantity
421,MLB4128653952,MLB29319,nan,C4PMLL001PK,Pandeirola Pandeiro Meia Lua Liverpool Profissional Rosa,1


In [31]:
print(df_stock_today.shape)
df_stock_today.sample()

(0, 5)


ValueError: a must be greater than 0 unless no samples are taken

#### Buscando Produtos

In [ ]:
# Buscando dados de produtos na tabela tiny_fulfillment
try:
    conn = psycopg2.connect(**db_config)

    sql_query = "SELECT * FROM tiny_fulfillment"
    df_codes = pd.read_sql(sql_query, conn)
except psycopg2.Error as e:
    # logger.error(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")
    print(f"Erro do psycopg2 ao consultar tiny_fulfillment: {e}")
    
except Exception as e:
    # logger.error(f"Erro ao consultar tabela tiny_fulfillment: {e}")
    print(f"Erro ao consultar tabela tiny_fulfillment: {e}")

finally:
    if conn is not None:
        conn.close()

df_codes['ml_code'] = df_codes['ml_code'].apply(lambda x: 'MLB' + str(x))
df_codes.rename(columns={'quantity': 'total_sales_quantity'}, inplace=True)
df_codes = df_codes.drop(['mcenter_id', 'created_at', 'updated_at'],axis=1)


C:\Users\couti\AppData\Local\Temp\ipykernel_9648\3478104386.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_codes = pd.read_sql(sql_query, conn)


In [ ]:
df_codes.sample()

,ml_inventory_id,ml_code,ml_sku,var_code,tiny_id,tiny_sku,ad_title,qtd_item
354,PAGJ50997,MLB1626396015,FULLCARIMBRAS0246,NaN,689455657,7898017033028,Jogos De Tabuleiro 4 Em 1 Carimbras 0246 Brinquedo Educativo,1


### Produtos + Dias disponíveis

In [ ]:
print(df_codes.shape)
print(df_stock_today.shape)

print(df_codes.columns)
print(df_stock_today.columns)

(872, 8)
(635, 5)
Index(['ml_inventory_id', 'ml_code', 'ml_sku', 'var_code', 'tiny_id',
       'tiny_sku', 'ad_title', 'qtd_item'],
      dtype='object')
Index(['ml_inventory_id', 'available_quantity_today', 'data', 'has_stock',
       'days_available'],
      dtype='object')


In [ ]:
prod_day = pd.merge(df_codes,df_stock_today,on='ml_inventory_id', how='inner')

prod_day.shape

(790, 12)

In [ ]:
prod_day['ml_inventory_id'].value_counts()
condf(prod_day,'ml_inventory_id','FSNB76403')

,ml_inventory_id,ml_code,ml_sku,var_code,tiny_id,tiny_sku,ad_title,qtd_item,available_quantity_today,data,has_stock,days_available
439,FSNB76403,MLB1943185424,FULLKIT2,NaN,746249012,RH-01SV,Kit Straplock Afinador Capotraste 6 Palhetas Porta Palheta,2,0,2023-11-27,False,0
440,FSNB76403,MLB1943185424,FULLKIT2,NaN,565665071,Palheta,Kit Straplock Afinador Capotraste 6 Palhetas Porta Palheta,6,0,2023-11-27,False,0
441,FSNB76403,MLB1943185424,FULLKIT2,NaN,565663877,PPalheta,Kit Straplock Afinador Capotraste 6 Palhetas Porta Palheta,1,0,2023-11-27,False,0
442,FSNB76403,MLB1943185424,FULLKIT2,NaN,509978212,Bracadeirakits,Kit Straplock Afinador Capotraste 6 Palhetas Porta Palheta,1,0,2023-11-27,False,0
443,FSNB76403,MLB1943185424,FULLKIT2,NaN,597415799,AFINADORKIT,Kit Straplock Afinador Capotraste 6 Palhetas Porta Palheta,1,0,2023-11-27,False,0


### Prod_Day + Total_sales

In [ ]:
print(df_total_sales.shape)
df_total_sales.sample()

(261, 6)


,ml_code,category_id,variation_id,seller_sku,title,total_sales_quantity
782,MLB3417715029,MLB72745,nan,C4NINJA03FT,"Cabo P10 X P10 Guitarra Baixo Santo Angelo Ninja 0,91 Metros",2


In [ ]:
df_sales = pd.merge(df_total_sales, prod_day, left_on=['ml_code','seller_sku'], right_on=['ml_code', 'ml_sku'], how='inner')
# x  = pd.merge(df_total_sales, prod_day, left_on=['ml_code','seller_sku'], right_on=['ml_code', 'ml_sku'], how='left')

# df_sales = df_sales.drop([], axis=1)

cols = ['ml_code', 'ml_sku', 'ml_inventory_id', 'tiny_id', 'tiny_sku', 'var_code', 'variation_id', 'title', 'total_sales_quantity', 'qtd_item','days_available', 'available_quantity_today', 'data']

df_sales = df_sales[cols]
print(df_total_sales.shape)
print(prod_day.shape)
# print(x.shape)
print(df_sales.shape)


(261, 6)
(790, 12)
(132, 13)


In [ ]:
df_sales.sample()

,ml_code,ml_sku,ml_inventory_id,tiny_id,tiny_sku,var_code,variation_id,title,total_sales_quantity,qtd_item,days_available,available_quantity_today,data
25,MLB1991060554,FULLBASSOCLA01,JSMU98097,729127565,7898942136054,NaN,nan,Correia Guitarra Violão Baixo Alça Basso Cla 01 Camurca Bege,5,1,10,10,2023-11-27


In [ ]:
df_sales.columns

Index(['ml_code', 'ml_sku', 'ml_inventory_id', 'tiny_id', 'tiny_sku',
       'var_code', 'variation_id', 'title', 'total_sales_quantity', 'qtd_item',
       'days_available', 'available_quantity_today', 'data'],
      dtype='object')

In [ ]:
# x = pd.merge(x, df_sales, how='outer', indicator=True)
# x = x[x['_merge'] == 'right_only']

# x

In [ ]:
condf(df_total_sales,'ml_code','MLB1992541482')

,ml_code,category_id,variation_id,seller_sku,title,total_sales_quantity
0,MLB1992541482,MLB278076,173552533267,FULLSG6687,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 85/15 Bronze Sg,5
648,MLB1992541482,MLB278076,173552533264,FULLSG6684,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 85/15 Bronze Sg,3


In [ ]:
condf(df_sales,'ml_code','MLB1992541482')

,ml_code,ml_sku,ml_inventory_id,tiny_id,tiny_sku,var_code,variation_id,title,total_sales_quantity,qtd_item,days_available,available_quantity_today,data
0,MLB1992541482,FULLSG6687,CLMF99895,730228363,7897626866874,0.12,173552533267,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 85/15 Bronze Sg,5,1,10,8,2023-11-27
73,MLB1992541482,FULLSG6684,LUHT04035,747847952,7897626866843,0.09,173552533264,Encordoamento 0.09 0.10 0.11 0.12 Violão Aço 85/15 Bronze Sg,3,1,10,19,2023-11-27


### Calculando métricas

In [ ]:
df_sales.sample()

,ml_code,ml_sku,ml_inventory_id,tiny_id,tiny_sku,var_code,variation_id,title,total_sales_quantity,qtd_item,days_available,available_quantity_today,data
83,MLB1814087324,FULLKIT01,FTGG16520,597415799,AFINADORKIT,NaN,nan,Kit 6 Palhetas + 1 Porta Palheta + 1 Capotraste + 1 Afinador,2,1,0,0,2023-11-27


In [ ]:
# media de produtos disponiveis no período
df_sales['media_prod_days_available'] = (df_sales['total_sales_quantity'] / df_sales['days_available'])
df_sales['media_prod_days_available'] = df_sales['media_prod_days_available'].fillna(0)

days = 30

# qtd de produtos a enviar no período, caso seja valor negativo produto está acima do esperado para envio(sobrando)
df_sales['period_send_fulfillment'] = np.ceil((df_sales['total_sales_quantity'] / df_sales['days_available'])* days - df_sales['available_quantity_today'])
df_sales['period_send_fulfillment'] = df_sales['period_send_fulfillment'].fillna(0)

# qtd de produtos a enviar hoje, caso seja valor negativo produto está acima do esperado para envio(sobrando)
df_sales['today_send_fulfillment'] = np.ceil((df_sales['total_sales_quantity'] / df_sales['days_available']) - df_sales['available_quantity_today'])
df_sales['today_send_fulfillment'] = df_sales['today_send_fulfillment'].fillna(0)

In [ ]:
df_sales.sample(2)

,ml_code,ml_sku,ml_inventory_id,tiny_id,tiny_sku,var_code,variation_id,title,total_sales_quantity,qtd_item,days_available,available_quantity_today,data,media_prod_days_available,period_send_fulfillment,today_send_fulfillment
68,MLB1729421535,IZZO3440PRETO,SYZS70682,741799625,7897626864801,Preto,69642779895,Pandeiro Preto Izzo 10 Polegada Abs Com Pele Preta - 3440,3,1,9,0,2023-11-27,0.333333,10.0,1.0
57,MLB2024265635,FULLERNIEBALLP02221,NIPX18741,730857747,749699122210,0.010,173664488134,Cordas Para Guitarra 09 10 11 Power Slink Niquel Ernie Ball,1,1,10,16,2023-11-27,0.100000,-13.0,-15.0


In [ ]:
x = 'MLB1629379059'
y = df_sales[df_sales['ml_code'] == x]
y

,ml_code,ml_sku,ml_inventory_id,tiny_id,tiny_sku,var_code,variation_id,title,total_sales_quantity,qtd_item,days_available,available_quantity_today,data,media_prod_days_available,period_send_fulfillment,today_send_fulfillment
21,MLB1629379059,FULLP10NINJA15FT,ZOQD52873,567508524,7899028808551,NaN,nan,Cabo P10 X P10 Santo Angelo Ninja 15ft - 4.57m,1,1,8,0,2023-11-27,0.125,4.0,1.0


In [ ]:
condf(df_sales,'ml_code','MLB1992541482')

Pergunta:

Caso em seja necessário enviar produtos de um kit e apenas um dos produtos estiver em falta, o que fazer?